# Graphing Runnables

In [1]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [34]:
pip show langchain

Name: langchain
Version: 0.3.20
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\users\hp\anaconda3\envs\longchain_envllm\lib\site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [35]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [37]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [38]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\HP\anaconda3\envs\longchain_envllm\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [39]:
string_parser = StrOutputParser()

In [10]:
pip install grandalf

  Using cached grandalf-0.8-py3-none-any.whl.metadata (1.7 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached grandalf-0.8-py3-none-any.whl (41 kB)
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.


In [40]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

In [41]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

# RunnableParallel

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [43]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [44]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\HP\anaconda3\envs\longchain_envllm\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [45]:
string_parser = StrOutputParser()

In [46]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [47]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [48]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup.\n2. Developing a simple Machine Learning model using Scikit-learn.\n3. Creating a basic web application using Django or Flask.'}

In [49]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [50]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 3.51 s


'1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin'

In [51]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 1.6 s


'1. Building a Web Scraper using BeautifulSoup and Requests.\n2. Developing a simple Machine Learning model using Scikit-learn.\n3. Creating a GUI application with Tkinter.'

In [52]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 78.1 ms
Wall time: 2.69 s


{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup and Requests\n2. Developing a Text-Based Adventure Game using Object-Oriented Programming\n3. Creating a Simple Machine Learning Model using Scikit-Learn'}

# Piping a RunnableParallel with Other Runnables

In [53]:
from langchain_core.runnables import RunnableParallel

In [54]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [55]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 500)

C:\Users\HP\anaconda3\envs\longchain_envllm\lib\site-packages\IPython\core\interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [56]:
string_parser = StrOutputParser()

In [57]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [58]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [59]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup\n2. Developing a simple Machine Learning application with Scikit-learn\n3. Creating a GUI-based Music Player using Tkinter'}

In [60]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | chat 
              | string_parser
             )

In [61]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

In [62]:
print(chain_time2.invoke({'programming language':'Python'}))

The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, as a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks to complete.

For the projects:

1. Building a Web Scraper using BeautifulSoup - If you work on it for about 2-3 hours a day, it might take you around 1-2 week

In [63]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            